# Pinecone Canopy with Anyscale

**Canopy** is a Software Development Kit (SDK) for AI applications. Canopy allows you to test, build and package Retrieval Augmented Applications with Pinecone Vector Database. 

This notebook introduces the quick start steps for working with Canopy and Anyscale Endpoints

## Prerequisites

install canopy library

In [ ]:
!pip install -qU canopy-sdk

In this example, Canopy uses Pinecone and Anyscale so we need to configure the related API keys.

To get Pinecone free trial API key and environment register or log into your Pinecone account in the [console](https://app.pinecone.io/). You can access your API key from the "API Keys" section in the sidebar of your dashboard, and find the environment name next to it.

You can find your free Anyscale API key [here](https://https://app.endpoints.anyscale.com/). 

In [ ]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["PINECONE_ENVIRONMENT"] = PINECONE_ENVIRONMENT
os.environ["ANYSCALE_API_KEY"]=ANYSCALE_API_KEY

Each record in this dataset represents a single page in Pinecone's documentation. Each row contains a unique id, the raw text of the page in markdown language, the url of the page as "source" and some metadata. 

## Pinecone Documentation Dataset

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

data = pd.read_parquet("https://storage.googleapis.com/pinecone-datasets-dev/pinecone_docs_ada-002/raw/file1.parquet")
data.head()

,id,text,source,metadata
0,728aeea1-1dcf-5d0a-91f2-ecccd4dd4272,# Scale indexes\n\n[Suggest Edits](/edit/scali...,https://docs.pinecone.io/docs/scaling-indexes,"{'created_at': '2023_10_25', 'title': 'scaling..."
1,2f19f269-171f-5556-93f3-a2d7eabbe50f,# Understanding organizations\n\n[Suggest Edit...,https://docs.pinecone.io/docs/organizations,"{'created_at': '2023_10_25', 'title': 'organiz..."
2,b2a71cb3-5148-5090-86d5-7f4156edd7cf,# Manage datasets\n\n[Suggest Edits](/edit/dat...,https://docs.pinecone.io/docs/datasets,"{'created_at': '2023_10_25', 'title': 'datasets'}"
3,1dafe68a-2e78-57f7-a97a-93e043462196,# Architecture\n\n[Suggest Edits](/edit/archit...,https://docs.pinecone.io/docs/architecture,"{'created_at': '2023_10_25', 'title': 'archite..."
4,8b07b24d-4ec2-58a1-ac91-c8e6267b9ffd,# Moving to production\n\n[Suggest Edits](/edi...,https://docs.pinecone.io/docs/moving-to-produc...,"{'created_at': '2023_10_25', 'title': 'moving-..."


## Init a Tokenizer


Many of Canopy's components are using tokenization, which is a process that splits text into tokens - basic units of text (like word or sub-words) that are used for processing. Therefore, Canopy uses a singleton `Tokenizer` object which needs to be initialized once.

In [ ]:
from canopy.tokenizer import Tokenizer, LlamaTokenizer
Tokenizer.initialize(tokenizer_class=LlamaTokenizer)

In [4]:
tokenizer = Tokenizer()
tokenizer.tokenize("Hello world!")

['▁Hello', '▁world', '!']

## Creating a KnowledgBase to store our data for search

The `KnowledgeBase` object is responsible for storing and indexing textual documents.

Once documents are indexed, the `KnowledgeBase` can be queried with a new unseen text passage, for which the most relevant document chunks are retrieved.

The `KnowledgeBase` holds a connection to a Pinecone index and provides a simple API to insert, delete and search textual documents.

The `KnowledgeBase`'s `upsert()` operation is used to index new documents, or update already stored documents. The `upsert` process splits each document's text into smaller chunks, transforms these chunks to vector embeddings, then upserts those vectors to the underlying Pinecone index. At Query time, the `KnowledgeBase` transforms the textual query text to a vector in a similar manner, then queries the underlying Pinecone index to retrieve the top-k most closely matched document chunks.

Here we create a `KnowledgeBase` with our desired index name: 

In [5]:
from canopy.knowledge_base import KnowledgeBase
from canopy.knowledge_base.record_encoder import AnyscaleRecordEncoder
INDEX_NAME = "PINECONE_INDEX_NA"
kb = KnowledgeBase(index_name=INDEX_NAME,
                  record_encoder=AnyscaleRecordEncoder())

In [6]:
from canopy.knowledge_base import list_canopy_indexes

if not any(name.endswith(INDEX_NAME) for name in list_canopy_indexes()):
    kb.create_canopy_index(indexed_fields=["title"], dimension=1024)

In [7]:
kb.connect()

> 💡 Note: a knowledge base must be connected to an index before executing any operation. You should call `kb.connect()` to connect  an existing index or call `kb.create_canopy_index(INDEX_NANE)` before calling any other method of the KB 

## Upsert data to our KnowledgBase

In [8]:
from canopy.models.data_models import Document
documents = [Document(**row) for _, row in data.iterrows()]

In [ ]:
from tqdm.auto import tqdm

batch_size = 5

for i in tqdm(range(0, len(documents), batch_size)):
    kb.upsert(documents[i: i+batch_size])

## Query the KnowledgeBase

Now we can query the knowledge base. The KnowledgeBase will use its default parameters like `top_k` to execute the query:

In [10]:
def print_query_results(results):
    for query_results in results:
        print('query: ' + query_results.query + '\n')
        for document in query_results.documents:
            print('document: ' + document.text.replace("\n", "\\n"))
            print("title: " + document.metadata["title"])
            print('source: ' + document.source)
            print(f"score: {document.score}\n")

In [ ]:
from canopy.models.data_models import Query
results = kb.query([Query(text="p1 pod capacity")])

print_query_results(results)

You can change the `top_k` parameter, to determine the number of top query results that will be returned and also to provide a [metadata filter](https://docs.pinecone.io/docs/metadata-filtering).

In [ ]:
from canopy.models.data_models import Query
results = kb.query([Query(text="p1 pod capacity",
                          #metadata_filter={"source": "https://docs.pinecone.io/docs/limits"},
                          top_k=2)])

print_query_results(results)

As you can see above, using the metadata filter we get results only from the "limits" page

## Query the Context Engine

`ContextEngine` is an object responsible for retrieving the most relevant context for a given query and token budget.  

While `KnowledgeBase` retrieves the full `top-k` structured documents for each query including all the metadata related to them, the context engine in charge of transforming this information to a "prompt ready" context that can later feeded to an LLM. To achieve this the context engine holds a `ContextBuilder` object that takes query results from the knowledge base and returns a `Context` object. The `ContextEngine`'s default behavior is to use a `StuffingContextBuilder`, which simply stacks retrieved document chunks in a JSON-like manner, hard limiting by the number of chunks that fit the `max_context_tokens` budget. More complex behaviors can be achieved by providing a custom `ContextBuilder` class.

In [13]:
from canopy.context_engine import ContextEngine
context_engine = ContextEngine(kb)

In [ ]:
import json

result = context_engine.query([Query(text="capacity of p1 pods", top_k=5)], max_context_tokens=512)

print(result.to_text(indent=2))
print(f"\n# tokens in context returned: {result.num_tokens}")

As you can see above, although we set `top_k=5`, context engine retreived only 3 results in order to satisfy the 512 tokens limit. Also, the documents in the context contain only the text and source and not all the metadata that is not necessarily needed by the LLM. 

## Knowledgeable chat engine

Now we are ready to start chatting with our data!

Canopy's `ChatEngine` is a one-stop-shop RAG-infused Chatbot. The `ChatEngine` wraps an underlying LLM such as Llama models, enhancing it by providing relevant context from the user's knowledge base. It also automatically phrases search queries out of the chat history and send them to the knowledge base.

In [15]:
from canopy.chat_engine import ChatEngine
from canopy.llm import BaseLLM, AnyscaleLLM
from canopy.chat_engine.query_generator import LastMessageQueryGenerator
chat_engine = ChatEngine(context_engine,
                         llm=AnyscaleLLM(),
                         query_builder=LastMessageQueryGenerator())

In [16]:
from typing import Tuple
from canopy.models.data_models import Messages, UserMessage, AssistantMessage

def chat(new_message: str, history: Messages) -> Tuple[str, Messages]:
    messages = history + [UserMessage(content=new_message)]
    response = chat_engine.chat(messages)
    assistant_response = response.choices[0].message.content
    return assistant_response, messages + [AssistantMessage(content=assistant_response)]

In [17]:
from IPython.display import display, Markdown

history = []
response, history = chat("What is the capacity of p1 pods?", history)
display(Markdown(response))

  According to the context, the capacity of p1 pods is approximately 1M vectors with 768 dimensions. This information is provided in the document titled "Limits" in the context.

Source: <https://docs.pinecone.io/docs/limits/>

In [18]:
history

[UserMessage(role=<Role.USER: 'user'>, content='What is the capacity of p1 pods?'),
 AssistantMessage(role=<Role.ASSISTANT: 'assistant'>, content='  According to the context, the capacity of p1 pods is approximately 1M vectors with 768 dimensions. This information is provided in the document titled "Limits" in the context.\n\nSource: <https://docs.pinecone.io/docs/limits/>')]

In [19]:
response, history = chat("And for what latency requirements does it fit?", history)
display(Markdown(response))

  According to the context, p1 pods are optimized for very low query latencies, with latencies of less than 100ms. This means that they are suitable for applications that require fast query responses, such as real-time analytics or dashboards.

Source: <https://docs.pinecone.io/docs/choosing-index-type-and-size/>

In contrast, p2 pods are optimized for greater query throughput with lower latency, with query latencies of less than 10ms for vectors with fewer than 128 dimensions. This makes them more suitable for applications with moderate to high query volumes, such as data-intensive analytics or machine learning workloads.

Source: <https://docs.pinecone.io/docs/p2-pods>

Therefore, the choice of pod type depends on the specific latency requirements of the application. If very low latencies are required, p1 pods may be a better fit, while if moderate to high query volumes are involved, p2 pods may be more suitable.

In [20]:
history

[UserMessage(role=<Role.USER: 'user'>, content='What is the capacity of p1 pods?'),
 AssistantMessage(role=<Role.ASSISTANT: 'assistant'>, content='  According to the context, the capacity of p1 pods is approximately 1M vectors with 768 dimensions. This information is provided in the document titled "Limits" in the context.\n\nSource: <https://docs.pinecone.io/docs/limits/>'),
 UserMessage(role=<Role.USER: 'user'>, content='And for what latency requirements does it fit?'),
 AssistantMessage(role=<Role.ASSISTANT: 'assistant'>, content='  According to the context, p1 pods are optimized for very low query latencies, with latencies of less than 100ms. This means that they are suitable for applications that require fast query responses, such as real-time analytics or dashboards.\n\nSource: <https://docs.pinecone.io/docs/choosing-index-type-and-size/>\n\nIn contrast, p2 pods are optimized for greater query throughput with lower latency, with query latencies of less than 10ms for vectors with 

> 💡 Note: Canopy calls the underlying LLM, providing both the user-provided chat history and a generated `Context` prompt. This might surpass the `ChatEngine`'s configured `max_prompt_tokens`. By default, the `ChatEngine` would truncate the oldest messages in the chat history to avoid exceeding this limit. This behavior in configurable, as explained in the [documentation](https://github.com/pinecone-io/canopy/blob/main/src/canopy/chat_engine/chat_engine.py)